In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

from numpy import pi, log, sin, cos, exp

from riptide.libffa import LibFFA, get_snr, downsample
from riptide.ffautils import get_period
from riptide import ffa_transform_1d, ffa_transform_2d, generate_signal

%load_ext autoreload
%autoreload 2

In [4]:
# def generate_signal(nsamp, period, phi0=0.0, ducy=0.01):
#     """ Generate a periodic signal with a von Mises profile.
#     Parameters:
#     -----------
#         nsamp : number of samples to generate
#         period: period in number of samples
#         phi0  : initial signal phase in periods
#         ducy  : duty cycle of the pulse
#     """
#     # von mises parameter
#     kappa = log(2.0) / (2.0 * sin(pi*ducy/2.0)**2)
    
#     phase_radians = (np.arange(nsamp, dtype=float) / period - phi0) * (2 * pi)
#     return exp(kappa*(cos(phase_radians) - 1.0))


# def ffa_transform(tseries, pnum):
#     """ Transform time series at given (integer) period number """
#     pnum = int(pnum)
#     nsamp = tseries.size
#     m = nsamp // pnum
#     x = tseries[:m*pnum]
#     out = np.zeros(shape=(m, pnum), dtype=np.float32)
#     LibFFA.py_transform(
#         x.astype(np.float32).reshape(m, pnum),
#         m,
#         pnum,
#         out
#         )
#     return out


# def get_width_trials(b, wmax=0.25, wtsp=1.5):
#     """ Generate the list of width trials """
#     widths = []
#     w = 1
#     wmax = max(1, wmax * b)
#     while w <= wmax:
#         widths.append(w)
#         w = max(w + 1, int(w * wtsp))
#     return np.asarray(widths)
    

# def get_snrs(trans, widths, varnoise=1.0, threads=1):
#     m, b = trans.shape
#     snrs = np.zeros(shape=(m, widths.size), dtype=np.float32)
#     LibFFA.get_snr_2d(trans, m, b, widths, widths.size, varnoise, threads, snrs)
#     return snrs


# def get_period_trials(shift, m, b):
#     """ Compute the exact period trial value associated with a given top-to-bottom shift """
#     return b + shift*b/(m*b - shift)

In [2]:
##### REAL DATA
fname = '/home/vince/work/pulsars/ffa/J1914+0845.64us.npy'
tsamp = 64e-6
dsfactor = 4
data = np.load(fname)

data = data[:data.size//dsfactor * dsfactor].reshape(-1, dsfactor).sum(axis=1)
data -= data.mean()
data /= data.std()
tsamp *= dsfactor
stdnoise = 1.0

#### FFA Transform
b = 859*2
m = data.size // b

### Block of data that actually gets transformed
s = data[:m*b]

In [91]:
### ARTIFICIAL DATA
b = 200 # bins
m = 200 # number of profiles

period_offset = 0.502 # value in [0, 1[
phi0 = 0.25 # initial phase in number of periods
ducy = 0.015
amplitude = 5.0
stdnoise = 1.0


# line index at which the FFA output should peak
ffa_lmax_naive = period_offset * m 
ffa_lmax_true = period_offset * m * b / (b + period_offset) 

# column index at which the FFA output should peak
ffa_cmax = phi0 * b 

#### Build von Mises pulse train
s = generate_signal(m*b, b + period_offset, phi0=phi0, ducy=ducy, amplitude=amplitude, stdnoise=stdnoise)
s -= s.mean()

In [92]:
#### FFA Transform
out = ffa_transform_1d(s, b)

### Compute S/N
snrs, widths = get_snr(out, stdnoise=stdnoise*m**0.5)

In [93]:
### Input and its FFA transform
plt.figure(figsize=(8,4), dpi=250)
plt.subplot(121)
plt.imshow(s.reshape(m, b), cmap=plt.cm.Greys, aspect='auto')
plt.xlabel('Phase bin')
plt.ylabel('Profile index')
plt.title('Input Data', fontsize=14)
#plt.grid(linestyle=':')

plt.subplot(122)
plt.imshow(out, cmap=plt.cm.Greys, aspect='auto')
#plt.plot(ffa_cmax, ffa_lmax_naive, 'rx', markersize=10)
#plt.plot(ffa_cmax, ffa_lmax_true, 'gx', markersize=10)
plt.xlabel('Phase bin')
plt.ylabel('Shift')
plt.title('Folding Transform', fontsize=14)
#plt.grid(linestyle=':')

plt.tight_layout()
plt.show()

In [15]:
plt.figure(figsize=(12,12), dpi=150)
plt.subplot(311)
plt.imshow(out.T, cmap=plt.cm.Greys, aspect='auto')
plt.ylabel('Phase bin')
plt.xlabel('Shift')
plt.grid(linestyle=':')

plt.subplot(312)
plt.imshow(snrs.T, aspect='auto')
plt.yticks(range(widths.size), widths, fontsize=12)
plt.xlabel('Shift')
plt.ylabel('Boxcar width trial (bins)')
plt.grid(linestyle=':')
plt.tight_layout()

plt.subplot(313)
plt.plot(snrs, linestyle=':', linewidth=0.8)
plt.plot(snrs.max(axis=1), color='k', linewidth=1.5)
plt.xlabel('Shift')
plt.ylabel('Best S/N')
plt.xlim(-0.5, m-0.5)
plt.grid(linestyle=':')
plt.tight_layout()
plt.show()


ibest = snrs.max(axis=1).argmax()
plt.figure(figsize=(8,4), dpi=150)
plt.plot(out[ibest] / stdnoise / m**0.5, linewidth=1.0)
plt.xlabel('Phase bin')
plt.ylabel('Normalized amplitude')
plt.title('Best Folded Profile')
plt.xlim(0, b-1)
plt.grid(linestyle=':')
plt.tight_layout()
plt.show()


# ### Period trials
# shifts = np.arange(m)
# period_trials = get_period_trials(shifts, m, b)

# plt.figure(figsize=(6, 4.5), dpi=150)
# plt.plot(shifts, period_trials - period_trials[0])
# plt.xlabel('Shift (bins)')
# plt.xlim(0, m-1)

# plt.ylabel('Period Trial - %d (bins)' % b)
# plt.ylim(0.0, period_trials[-1] - period_trials[0])
# plt.grid()
# plt.tight_layout()
# plt.show()

In [29]:
### S/N BENCHMARK RESULTS

# 8192 x 512b input, runtimes in ms, FFA: 20.2ms
# threads, runtime, speedup
# 1 14.7 
# 2 7.41  1.98
# 3 5.07  2.90
# 4 4.1   3.58
# 6 4.04  3.63
# 8 3.58  4.11
# tSNR / tFFA (1 thread) = 0.72


# 1024 x 256b input, runtimes in us, FFA: 719us
# threads, runtime, speedup
# 1 936 
# 2 481 1.94
# 3 333 2.81 
# 4 276 3.39 
# 6 288 
# 8 256
# tSNR / tFFA (1 thread) = 1.30


# 256 x 1024b input, runtimes in us, FFA: 528us
# threads, runtime, speedup
# 1 1040
# 2 533  1.95
# 3 377  2.76
# 4 298  3.48
# 6 296
# 8 290
# tSNR / tFFA (1 thread) = 1.97

In [ ]:
4.20135042922265e-07